# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [3]:
import pandas as pd

def clean_customer_data(df):
    # Standardize column names
    df.columns = (
        df.columns.str.strip().str.lower().str.replace(" ", "_")
    )
    
    # Fix gender
    if 'gender' in df.columns:
        df['gender'] = df['gender'].str.strip().str.lower().replace({
            'femal': 'f', 'female': 'f', 'f': 'f',
            'male': 'm', 'm': 'm'
        }).str.upper()
    
    # Fix state
    if 'state' in df.columns:
        df['state'] = df['state'].replace({
            'AZ': 'Arizona', 'Cali': 'California', 'WA': 'Washington'
        })
    
    # Fix education
    if 'education' in df.columns:
        df['education'] = df['education'].replace({'Bachelors': 'Bachelor'})

    # Fix customer lifetime value
    if 'customer_lifetime_value' in df.columns:
        df['customer_lifetime_value'] = (
            df['customer_lifetime_value']
            .astype(str)
            .str.replace('%', '', regex=False)
        )
        df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

    # Fix vehicle class
    if 'vehicle_class' in df.columns:
        df['vehicle_class'] = df['vehicle_class'].replace({
            'Luxury SUV': 'Luxury', 'Luxury Car': 'Luxury', 'Sports Car': 'Luxury'
        })

    # Fix number_of_open_complaints
    if 'number_of_open_complaints' in df.columns:
        df['number_of_open_complaints'] = (
            df['number_of_open_complaints']
            .astype(str)
            .str.split("/")
            .str[1]
        )
        df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce')
    
    return df


In [9]:
# Load the files
file3 = pd.read_csv("file3.csv")
file4 = pd.read_csv("file4.csv")
file5 = pd.read_csv("file5.csv")

# Clean each dataset
file3_clean = clean_customer_data(file3)
file4_clean = clean_customer_data(file4)
file5_clean = clean_customer_data(file5)

# Combine into one DataFrame
combined_df = pd.concat([file3_clean, file4_clean, file5_clean], ignore_index=True)

# Optional: save the cleaned data
combined_df.to_csv("combined_cleaned_customer_data.csv", index=False)


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [11]:
marketing_df = pd.read_csv("marketing_customer_analysis_clean.csv")
marketing_df.columns = marketing_df.columns.str.strip().str.lower().str.replace(" ", "_")


In [16]:
revenue_by_channel = marketing_df.pivot_table(
    index='sales_channel',
    values='total_claim_amount',
    aggfunc='sum'
).round(2)

print(" Total Revenue by Sales Channel:")
print(revenue_by_channel)


 Total Revenue by Sales Channel:
               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


In [18]:
clv_by_gender_education = marketing_df.pivot_table(
    index='gender',
    columns='education',
    values='customer_lifetime_value',
    aggfunc='mean'
).round(2)

print("\n Average Customer Lifetime Value by Gender & Education:")
print(clv_by_gender_education)



 Average Customer Lifetime Value by Gender & Education:
education  Bachelor  College   Doctor  High School or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [17]:
# Your code goes here
# Ensure date column is in datetime format
marketing_df['effective_to_date'] = pd.to_datetime(marketing_df['effective_to_date'], errors='coerce')

# Create 'month' column
marketing_df['month'] = marketing_df['effective_to_date'].dt.month_name()

# Group and reshape in long format
complaints_summary = (
    marketing_df.groupby(['policy_type', 'month'])['number_of_open_complaints']
    .sum()
    .reset_index(name='total_complaints')
)

print("\n Complaints by Policy Type and Month (Long Format):")
print(complaints_summary.head())



 Complaints by Policy Type and Month (Long Format):
      policy_type     month  total_complaints
0  Corporate Auto  February        385.208135
1  Corporate Auto   January        443.434952
2   Personal Auto  February       1453.684441
3   Personal Auto   January       1727.605722
4    Special Auto  February         95.226817
